**Importando bibliotecas**

In [21]:
import pandas as pd
import seaborn as sns
import pickle

import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE


**Função para eliminar valores NANS, transformar valores qualitativos em quantitativos e separar features e labels**

In [22]:
def tratar_separar_dados(df):
    """  
   Input:
       df: pandas DataFrame com dados importados
   Output:
       dados_x: pandas DataFrame com dados do X (features) 
       dados_y: pandas DataFrame com dados do Y (label)
   """
    ###Prencher os valores NANs
    df = df.fillna(method='ffill')
        
    ######Transformar a coluna SEX em quantitativo
    df.loc[(df['sex'] == 'M',"sex")] = 1
    df.loc[(df['sex'] == 'm',"sex")] = 1
    
    df.loc[(df['sex'] == 'F',"sex")] = 0
    df.loc[(df['sex'] == 'f',"sex")] = 0
    
    df['sex'] = df['sex'].astype(int)
    ######
    
    ##plotar gráfico para ver correlação das features com o label
    #sns.heatmap(df.corr(), annot=True)
    
    ###Obtendo dados do y (label)
    dados_y = df['sex']
    ###obtendo dados do x (features)
    dados_x = df.drop(['sex'],axis=1)
    
    return dados_x, dados_y

**Essa função tem como objetivo normalizar os dados das features(x)**

In [23]:
def normalizar_dados (dados_x):
    """
    Input: 
        dados_x: pandas Dataframe desnormalizado do x
    Output: 
        dados_x_normalizado: array com dados do x normalizados
    """
    ##Normalização dos dados do x
    scaler = StandardScaler()
    scaler.fit(dados_x)
    dados_x_normalizado = scaler.transform(dados_x)
    
    ##Salvando o scaler utilizado nos dados de treinamento
    scaler_filename = "scaler.save"
    joblib.dump(scaler, scaler_filename) 
    ###
    
    return dados_x_normalizado

**Função para separar dados de treinamento e teste**

In [24]:
def separar_dados_treinamento_teste(dados_x_norm, dados_y ,porcentagem):
    """
   Input:
       dados_x_norm: pandas DataFrame com dados de treinamento (x) normalizados
       dados_y: pandas DataFrame com dados de treinamento(y)
       porcentagem: inteiro com a porcetagem de dados de treinamento

   Output:
       dados_treinamento_x: array com dados do X de treinamento 
       dados_teste_x: array com dados do X de teste 
       dados_treinamento_y: array com dados do y de treinamento 
       dados_teste_y: array com dados do y de teste
    """
    ##Obtendo o tamanho do dataframe importado
    tamanho_df = len(dados_x_norm)
    ##Obtendo quantidade de amostras que equivalem ao argumento porcentagem setado
    porcentagem_treinamento = int((tamanho_df*porcentagem)/100)
    ##Obtendo os valores de treinamento e teste do X
    dados_treinamento_x = dados_x_norm.iloc[0:porcentagem_treinamento]
    dados_teste_x = dados_x_norm.iloc[porcentagem_treinamento:tamanho_df]
    ##Obtendo os valores de treinamento e teste do Y
    dados_treinamento_y = dados_y.iloc[0:porcentagem_treinamento]
    dados_teste_y = dados_y.iloc[porcentagem_treinamento:tamanho_df]
   
    return dados_treinamento_x, dados_teste_x, dados_treinamento_y, dados_teste_y

**Essa função tem objetivo diminuir a dimensionalidade dos dados.**

In [25]:
def diminuir_dimensionalidade(dados_x_norm):
    """
   Input:
       dados_x_norm: pandas DataFrame com dados importados e normalizados do X
   Output:
       x_pca: pandas DataFrame do dataFrame do X com dimensões reduzidas
  """ 
    ##Aplicação do método PCA nos dados do x
    pca = PCA(n_components=14)
    pca.fit(dados_x_norm)
    x_pca = pd.DataFrame(pca.transform(dados_x_norm))
    ###
    
    return x_pca

**Essa função tem como objetivo de encontrar os melhores parâmetros para o modelo do Random Forest**

In [26]:
def encontrar_melhores_parametros(x_train,y_train):
    """
    Input: 
        x_train: lista com os valores de x (features)
        y_train: lista com os valores de y (label)
    Output: 

    """
    ##Criação de uma random forest
    rfc=RandomForestClassifier(random_state=42)

    ##Definição do espaço de busca
    param_grid = { 
        'n_estimators': [50, 200],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [2,4,5,6,7,8,10],
        'random_state' :[0,10]
    }
    
    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
    CV_rfc.fit(x_train, y_train)
    
    print(CV_rfc.best_params_)


**Essa função tem como objetivo balancear a quantidade de amostras das classes (labels)**

In [27]:
def balanceamento_das_amostras(x_train,y_train):
    """
    Input: 
        x_train: lista com os valores de x (features)
        y_train: lista com os valores de y (label)
    Output: 

    """
    oversample = SMOTE()
    x_bal, y_bal = oversample.fit_resample(x_train, y_train)
    
    y_bal = pd.DataFrame(y_bal,columns=['class'])
    print(sum(y_bal['class'] == 0))
    print(sum(y_bal['class'] == 1))

    return x_bal,y_bal

**Essa função tem como objetivo treinar modelos com os dados importados e devidamente tratados**

In [28]:
def treinar_modelo(x_train,y_train):
    """
    Input: 
        x_train: lista de valores de treinamneto do x (features)
        y_train: lista de valores de treinamento do y (label)
    Output: 
        Pickle: modelo treinado
    """
    ##Modelo MLPClassifier 
    #clf = MLPClassifier(random_state=10,hidden_layer_sizes=(50),tol=0.000001, verbose=False, max_iter=1550,solver='adam').fit(x_train, y_train)
    #clf.fit(x_train,y_train)
    
    ##Modelo Random Forest
    random_forest = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=10)
    random_forest.fit(x_train,y_train)
    return random_forest

**Função principal**

In [29]:
if __name__ == '__main__':
    
    ##Ler os dados dos pacientes disponíveis no csv 
    df_importado = pd.read_csv('test_data_CANDIDATE.csv',header=0)
    df_importado = df_importado.set_index(['index'])
    
    ##Transformando os valores qualitativos em quantitativos (sex), remoção de features, e separando as features e label.
    dados_x, dados_y = tratar_separar_dados(df_importado)
    
    ##Normalizando os dados das features (x)
    dados_x_norm = pd.DataFrame(normalizar_dados(dados_x))

    ##Diminuindo a dimensionalidade do problema 
    #pca_x = diminuir_dimensionalidade(dados_x_normalizado)
    
    ##Separar dados de treinamento e teste para criação de modelo
    train_x, test_x, train_y, test_y = separar_dados_treinamento_teste(dados_x_norm,dados_y,80)

    ##Transformando dataFrames em listas
    train_x = train_x.values.tolist()
    train_y = train_y.to_list()
    test_x = test_x.values.tolist()
    test_y = test_y.to_list()
    
    ##Encontrar melhores parametros para o modelo
    #encontrar_melhores_parametros(train_x, train_y)
    
    ##Balanceado as amostras
    #x,y = balanceamento_das_amostras(train_x,train_y)
        
    ##Treinar modelo 
    modelo = treinar_modelo(train_x,train_y)

    ##Avaliando a acurácia do modelo treinado
    print(modelo.score(test_x,test_y))

0.8275862068965517


**Salvar modelo treinado**

In [30]:
filename = 'modelo_treinado.sav'
pickle.dump(modelo, open(filename, 'wb'))